##  NQDM: Introduction

In this tutorial, we will see some examples of how to get rid of some code smells in your program with minimal effort using NQDM.
If you followed the setup instructions in README.md, you are ready to go!

First things first, let's import the modules:

In [1]:
from nqdm import nqdm
import numpy as np
import pandas as pd

### Level 1: Simple iteration

If input is a number, it is a length variable and interpreted as integer.

In [2]:
lenWeeks = 52
lenDays = 365
winterMonths = ["december", "january", "february"]

len_1 = lenDays / lenWeeks
len_2 = len(winterMonths)

for i in nqdm(len_1, desc="Days per week"):
  print(i)

for i in nqdm(len_2, desc="Months in winter"):
  print(i)

Months in winter: 100%|██████████| 3/3 [00:00<00:00, 5208.16it/s]



0
1
2
3
4
5
6


0
1
2


If input is a list or Numpy array, it is interpreted as iterable.

In [3]:
arg_1 = [1, 2, 3]
arg_2 = np.array([4, 5, 6])

for i in nqdm(arg_1, desc="1, 2, 3"):
  print(i)

for i in nqdm(arg_2, desc="4, 5, 6"):
  print(i)

4, 5, 6: 100%|██████████| 3/3 [00:00<00:00, 7534.68it/s]



1
2
3


4
5
6


Strings, too, are iterable objects, since they are basically list of chars.

In [4]:
arg_1 = list("abc")
arg_2 = "def"

for i in nqdm(arg_1, desc="a, b, c"):
  print(i)

for i in nqdm(arg_2, desc="d, e, f"):
  print(i)

d, e, f: 100%|██████████| 3/3 [00:00<00:00, 3521.67it/s]



a
b
c


d
e
f


Dictionaries and Pandas Dataframes are interpreted as list of key-value pairs, which can be iterated as well.

In [5]:
arg_1 = {"name": "aaron", "age": 50}
arg_2 = pd.Series([4, 87, 90], index=["peppers", "eggplants", "cucumbers"])

for i in nqdm(arg_1, desc="person data"):
  print(i)
  
for i in nqdm(arg_2, desc="grocery inventory"):
  print(i)

grocery inventory: 100%|██████████| 3/3 [00:00<00:00, 4539.29it/s]



{'name': 'aaron'}
{'age': 50}


{'peppers': 4}
{'eggplants': 87}
{'cucumbers': 90}


For a more detailled explanation, please read the following markdown:

### Working With Different Data Types

There are three types of arguments: 
1. constant: returns `0, ..., constant-1`
2. iterable: returns `iterable[0], ..., iterable[-1]`
3. hashable: returns `{keys[0]: values[0]}, ..., {keys[-1]: values[-1]}`

A more detailled list of available arguments and return values:

| Argument | Type of Argument                | Returns                                                   |
|----------|---------------------------------|-----------------------------------------------------------|
| constant | int, float, double              | range(int(constant))                                      |
| iterable | numpy.ndarray, range, list, str | list(iterable)                                            |
| hashable | pandas.core.series.Series       | [{k: v} for k, v in zip(hashable.index, hashable.values)] |
| hashable | dict                            | [{k: v} for k, v in hashable.items()]                     |


### Benchmarking

If we want to actually track the entire progress, we can also use `<iterator>.plot_history()` function

In [6]:
itr = nqdm(5*10**4)

for i in itr:
    for j in range(i):
        pass

itr.plot_history()

  4%|▍         | 2119/50000 [00:00<00:05, 8496.48it/s]

100%|██████████| 50000/50000 [01:16<00:00, 652.24it/s] 


### Level 2: Customized Iteration

##### Tip 1: Don't Slice Lists Inside Iterators, Do This Instead

In [7]:
numbers = list(range(256))
chars = list(map(chr, numbers))

# for i in chars[65:75]:
#     print(i)

# for i in chars[97:107]:
#     print(i)

itr = nqdm(chars, desc="letters")

for i in itr[65:75]:
    print(i)

for i in itr[97:107]:
    print(i)

letters: 100%|██████████| 10/10 [00:00<00:00, 28571.55it/s]



A
B
C
D
E
F
G
H
I
J


a
b
c
d
e
f
g
h
i
j


It also works using `maxiter=<number of iterations>`, but we will see more on this feature in the next section.

In [8]:
itr = nqdm(list("abcdef"), maxiter=3, desc="letters")

for i in itr:
    print(i)

letters:  50%|█████     | 3/6 [00:00<00:00, 2744.36it/s]



a
b
c


##### Tip 2: Don't Use `random.shuffle()`, Do This Instead

You can define an iterator object which randomizes the order of elements each time it is used to iterate, using `random=True`. 
Basically, it makes a set of tuples out of list of tuples.

In [9]:
old_list = list("abcdef")

# random.shuffle(old_list)
# for i in old_list:
#     print(i)

# random.shuffle(old_list)
# for i in old_list:
#     print(i) 

itr = nqdm(old_list, random=True, desc="letters")

for i in itr:
    print(i)

for i in itr:
    print(i)

letters: 100%|██████████| 6/6 [00:00<00:00, 6904.20it/s]



f
a
d
b
c
e


f
a
d
c
b
e


Using enum at the same time, you can track the original position of arguments: `enum=True`. 
So it makes list of tuples out of sets of tuples. You can get the previous order later using built-in function `sorted()`.

In [10]:
numbers = list(range(256))
chars = list(map(chr, numbers))

# enumerated_list = list(enumerate(chars))[65:75]
# random.shuffle(enumerated_list)

# for i in enumerated_list:
#     print(i)

itr = nqdm(chars, random=True, enum=True, desc="enumerated random letters")
for i in itr[65:75]:
    print(i)

# sort everything again
for i in sorted(itr[65:75]):
    print(i)

enumerated random letters: 100%|██████████| 10/10 [00:00<00:00, 16131.94it/s]



(67, 'C')
(70, 'F')
(68, 'D')
(71, 'G')
(74, 'J')
(65, 'A')
(73, 'I')
(66, 'B')
(69, 'E')
(72, 'H')


(65, 'A')
(66, 'B')
(67, 'C')
(68, 'D')
(69, 'E')
(70, 'F')
(71, 'G')
(72, 'H')
(73, 'I')
(74, 'J')


The key difference between slicing and using maxiter is when you set `random=True`. 
* Using maxiter=k, we choose k random elements. 
* Slicing k many elements of an nqdm instance, we choose a random permutation of the first k elements.

In [11]:
itr_slicing = nqdm(np.arange(25), random=True, desc="slicing")[:5]
itr_maxiter = nqdm(np.arange(25), random=True, maxiter=5, desc="maxiter")

for i in itr_slicing:
    print(i)

for i in itr_maxiter:
    print(i)

slicing: 0it [00:00, ?it/s]


maxiter:  20%|██        | 5/25 [00:00<00:00, 2708.45it/s]



3
0
1
4
2


6
10
12
17
1


##### Tip 3: Don't Write Multiple Loops, Do This Instead

Instead of defining multiple progress bars and multiple loops, you can iterate over multiple lists / dicts.

In [12]:
list_1 = ["black", "red", "blue"]
list_2 = ["aluminium", "iron", "wooden"]
list_3 = ["pencil", "pen", "eraser"]

# for i3 in list_3:
#     for i2 in list_2:
#         for i1 in list_1:
#             print(i1 + " " + i2 + " " + i3)

for i in nqdm(list_1, list_2, list_3):
    print(" ".join(i))


100%|██████████| 27/27 [00:00<00:00, 56257.43it/s]



black aluminium pencil
red aluminium pencil
blue aluminium pencil
black iron pencil
red iron pencil
blue iron pencil
black wooden pencil
red wooden pencil
blue wooden pencil
black aluminium pen
red aluminium pen
blue aluminium pen
black iron pen
red iron pen
blue iron pen
black wooden pen
red wooden pen
blue wooden pen
black aluminium eraser
red aluminium eraser
blue aluminium eraser
black iron eraser
red iron eraser
blue iron eraser
black wooden eraser
red wooden eraser
blue wooden eraser


##### Tip 4: Don't Write Multiple Loops Over A Nested List, Do This Instead 

You can also iterate over a higher-dimensional list using `depth={# levels to flatten}`.

In [13]:
list_of_list_of_lists = np.arange(8).reshape(2, 2, 2)

for list_of_lists in nqdm(list_of_list_of_lists, depth=0):
  print(list_of_lists)

# for list_of_lists in list_of_list_of_lists:
#   for lists in list_of_lists:
#     print(lists)

for arr in nqdm(list_of_list_of_lists, depth=1):
  print(arr)

# for list_of_lists in list_of_list_of_lists:
#   for lists in list_of_lists:
#     for item in lists:
#       print(lists)

for arr in nqdm(list_of_list_of_lists, depth=2):
  print(arr)


100%|██████████| 2/2 [00:00<00:00, 786.55it/s]

100%|██████████| 4/4 [00:00<00:00, 5864.11it/s]

100%|██████████| 8/8 [00:00<00:00, 30643.32it/s]



[[0 1]
 [2 3]]
[[4 5]
 [6 7]]


[0 1]
[2 3]
[4 5]
[6 7]


0
1
2
3
4
5
6
7


Not only lists but also dicts are suitable for deeper iterations.

In [14]:
list_of_dict_of_lists = [{"a": [0, 1], "b": [2, 3]}, {"a": [4, 5], "b": [6, 7]}]

# for dict_of_lists in list_of_dict_of_lists:
#   for lists in dict_of_lists.values():
#     for item in lists:
#       print(item)

for elem in nqdm(list_of_dict_of_lists, depth=2):
  print(elem)


100%|██████████| 8/8 [00:00<00:00, 31625.29it/s]



0
1
2
3
4
5
6
7


But if multiple arguments are given, it is better to specify the depth of each argument: `depth=[# levels of arg 1, …, # levels of arg n]`.

In [15]:
list_1_complex = [["alpha", "beta"], ["gamma", "delta"], ["epsilon", "zeta"], ["eta", "theta"]]
list_2_complex = [[{"alpha": 1}, {"beta": 2}, {"gamma": 3}, {"delta": 4}], [{"epsilon": 5}, {"zeta": 6}, {"eta": 7}, {"theta": 8}]]

# for list_2_complex_1 in list_2_complex:
#   for list_2_complex_2 in list_2_complex_1:
#     for list_2_complex_3 in list_2_complex_2.values():
#       for list_1_complex_1 in list_1_complex:
#         for list_1_complex_2 in list_1_complex_1:
#           print([list_1_complex_2, list_2_complex_3])


for elems in nqdm(list_1_complex, list_2_complex, depth=[1, 3]):
  print(elems)

100%|██████████| 64/64 [00:00<00:00, 18566.57it/s]



['alpha', 1]
['beta', 1]
['gamma', 1]
['delta', 1]
['epsilon', 1]
['zeta', 1]
['eta', 1]
['theta', 1]
['alpha', 2]
['beta', 2]
['gamma', 2]
['delta', 2]
['epsilon', 2]
['zeta', 2]
['eta', 2]
['theta', 2]
['alpha', 3]
['beta', 3]
['gamma', 3]
['delta', 3]
['epsilon', 3]
['zeta', 3]
['eta', 3]
['theta', 3]
['alpha', 4]
['beta', 4]
['gamma', 4]
['delta', 4]
['epsilon', 4]
['zeta', 4]
['eta', 4]
['theta', 4]
['alpha', 5]
['beta', 5]
['gamma', 5]
['delta', 5]
['epsilon', 5]
['zeta', 5]
['eta', 5]
['theta', 5]
['alpha', 6]
['beta', 6]
['gamma', 6]
['delta', 6]
['epsilon', 6]
['zeta', 6]
['eta', 6]
['theta', 6]
['alpha', 7]
['beta', 7]
['gamma', 7]
['delta', 7]
['epsilon', 7]
['zeta', 7]
['eta', 7]
['theta', 7]
['alpha', 8]
['beta', 8]
['gamma', 8]
['delta', 8]
['epsilon', 8]
['zeta', 8]
['eta', 8]
['theta', 8]


##### Tip 5: Don't Switch The For Loops, Do This Instead

Using  `order = 'first' | 'last' | [<order of iteration of arg 1>, …, <order of iteration of arg n>]`, you can customize the iteration order.

In [16]:
# for dim1 in range(2):
#     for dim2 in range(2):
#         for dim3 in range(2):
#             print([dim1, dim2, dim3])

for i in nqdm(2, 2, 2, order = "last"):
    print(i)

# for dim3 in range(2):
#     for dim2 in range(2):
#         for dim1 in range(2):
#             print([dim1, dim2, dim3])

for i in nqdm(2, 2, 2, order = "first"):
    print(i)

# for dim2 in range(2):
#     for dim1 in range(2):
#         for dim3 in range(2):
#             print([dim1, dim2, dim3])

for i in nqdm(2, 2, 2, order = [1, 2, 0]):
    print(i)

0it [00:00, ?it/s]

100%|██████████| 8/8 [00:00<00:00, 14788.20it/s]


[0, 0, 0]
[0, 0, 1]
[0, 1, 0]
[0, 1, 1]
[1, 0, 0]
[1, 0, 1]
[1, 1, 0]
[1, 1, 1]


[0, 0, 0]
[1, 0, 0]
[0, 1, 0]
[1, 1, 0]
[0, 0, 1]
[1, 0, 1]
[0, 1, 1]
[1, 1, 1]


[0, 0, 0]
[0, 0, 1]
[1, 0, 0]
[1, 0, 1]
[0, 1, 0]
[0, 1, 1]
[1, 1, 0]
[1, 1, 1]
